In [1]:
import torch
import tokenization
import parse_pdb
import pandas as pd

In [2]:
parse_pdb.load_pdb(['protein'], data_loc='./data/pdb_seqres.txt')

['>100d_A', 'mol:na', 'length:10', '', 'DNA/RNA', "(5'-R(*CP*)-D(*CP*GP*GP*CP*GP*CP*CP*GP*)-R(*G)-3')\n"]
['CCGGCGCCGG\n']


IndexError: list index out of range

In [2]:
type(lambda x: x)

function